In [ ]:
!pip install -q langchain-google-genai langchain-core langchain-community requests \
    google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


GEMINI KEY

In [ ]:

import os, getpass
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass.getpass("Paste your Gemini API key: ")
assert os.environ["GOOGLE_API_KEY"], "No Gemini API key set."


Paste your Gemini API key: ··········


INIT GEMI

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI

# Use a cheap, fast model; switch to "gemini-1.5-pro" if you want stronger reasoning.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)


WEATHER

In [ ]:

from langchain_core.tools import tool
import requests, json

                          ######################### Weather code mapping from Open-Meteo docs (condensed)####################################
WEATHER_CODES = {
    0:"Clear", 1:"Mainly clear", 2:"Partly cloudy", 3:"Overcast",
    45:"Fog", 48:"Depositing rime fog",
    51:"Light drizzle", 53:"Moderate drizzle", 55:"Dense drizzle",
    56:"Freezing drizzle (light)", 57:"Freezing drizzle (dense)",
    61:"Slight rain", 63:"Moderate rain", 65:"Heavy rain",
    66:"Freezing rain (light)", 67:"Freezing rain (heavy)",
    71:"Slight snow", 73:"Moderate snow", 75:"Heavy snow",
    77:"Snow grains",
    80:"Rain showers (slight)", 81:"Rain showers (moderate)", 82:"Rain showers (violent)",
    85:"Snow showers (slight)", 86:"Snow showers (heavy)",
    95:"Thunderstorm (slight/moderate)", 96:"Thunderstorm w/ slight hail", 99:"Thunderstorm w/ heavy hail"
}

@tool
def get_weather(city: str) -> str:
    """
    Return current weather for a city using the free Open-Meteo API.
    Input: plain city name (e.g., "Hyderabad").
    Output: JSON string with city, country, temp (°C), humidity (%), wind (m/s), condition.
    """
    if not city or not city.strip():
        return "City name is empty."

    # 1) Geocode city -> lat/lon
    g = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1, "language": "en"}
    )
    g.raise_for_status()
    gjs = g.json()
    if not gjs.get("results"):
        return f"City '{city}' not found."

    r0 = gjs["results"][0]
    lat, lon = r0["latitude"], r0["longitude"]
    name, country, tz = r0.get("name"), r0.get("country"), r0.get("timezone", "auto")

    # 2) Current weather
    w = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": lat, "longitude": lon, "timezone": tz,
            "current": "temperature_2m,relative_humidity_2m,wind_speed_10m,weather_code"
        }
    )
    w.raise_for_status()
    wj = w.json()
    cur = wj.get("current", {})

    code = int(cur.get("weather_code", -1)) if cur.get("weather_code") is not None else -1
    desc = WEATHER_CODES.get(code, "Unknown")

    payload = {
        "city": name, "country": country, "timezone": tz,
        "latitude": lat, "longitude": lon,
        "temperature_c": cur.get("temperature_2m"),
        "humidity_percent": cur.get("relative_humidity_2m"),
        "wind_speed_mps": cur.get("wind_speed_10m"),
        "condition": desc,
    }
    return json.dumps(payload)


PACKING ITEMS

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
!pip install -q langchain-google-genai langchain-core langchain-community requests \
    google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 faiss-cpu


# Example FAQ data
faq_text = """
Packing Tips:
- Always carry a light jacket, even in warm cities.
- For rainy places, carry an umbrella or raincoat.
- Wear comfortable shoes for walking.

Travel Safety:
- Keep copies of your documents in case of loss.
- Avoid isolated areas at night.
- Stay hydrated in hot weather.

Seasonal Advice:
- Summer: Light clothes, sunscreen, hat.
- Winter: Layered clothing, gloves, thermal wear.
- Monsoon: Waterproof shoes, quick-dry clothes.
"""

# Create chunks for embedding
docs = [Document(page_content=faq_text)]
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.split_documents(docs)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 48.0 MB/s eta 0:00:00


AI AGENT + RAG

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

@tool
def travel_faq(query: str) -> str:
    """Search travel FAQ for advice."""
    results = retriever.get_relevant_documents(query)
    if not results:
        return "No advice found."
    return "\n".join([r.page_content for r in results])

tools = [get_weather, travel_faq]

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are TravelMate, an AI agent that answers travel questions and fetches real-time weather. "
     "If user asks about weather, call 'get_weather'. "
     "If user asks for packing tips or travel safety, call 'travel_faq'. "
     "Always give short, practical answers."),
    # Few-shot example
    ("human", "I'm going to Mumbai in July, what should I pack?"),
    ("ai", "July in Mumbai is monsoon season. Carry a raincoat, umbrella, waterproof shoes, and light clothes."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
print("Ask about weather or travel advice (type 'exit' to quit).")
while True:
    q = input("\nYou: ").strip()
    if q.lower() in ("exit", "quit"):
        break
    out = agent_executor.invoke({"input": q})
    print("\nBot:", out["output"])


Ask about weather or travel advice (type 'exit' to quit).

You: hey tomm going to madurai can  tell wheter


> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Madurai'}`


{"city": "Madurai", "country": "India", "timezone": "Asia/Kolkata", "latitude": 9.919, "longitude": 78.11953, "temperature_c": 31.6, "humidity_percent": 55, "wind_speed_mps": 11.6, "condition": "Overcast"}Madurai weather: 31.6°C, overcast, 55% humidity, 11.6 m/s wind.

> Finished chain.

Bot: Madurai weather: 31.6°C, overcast, 55% humidity, 11.6 m/s wind.


KeyboardInterrupt: Interrupted by user